# Email Scored Scripts to Students (Enhanced)

Prepare and send personalized emails with each student's scored script attached.

**Enhancements:**
- ✅ Comprehensive SMTP configuration with connection testing
- ✅ Enhanced email template system with HTML formatting
- ✅ Dry-run mode for safe testing before actual sending
- ✅ Progress tracking with detailed success/failure reporting
- ✅ Email validation and error recovery
- ✅ Performance report integration
- ✅ Comprehensive distribution reporting and audit trails

This notebook:
- Reads `details_score_report.xlsx` to map student IDs to final marks.
- Attaches the corresponding scored PDF for each student and sends an email using SMTP.
- Uses `../smtp.config` for SMTP credentials and server settings — update this file before sending.
- Includes a customizable subject and body template; update `subject` and `body_template` to match your messaging.
- Displays a progress bar while sending; test with a small subset before a full run to verify formatting and delivery.

Note: Ensure SMTP settings (server, port, email, password) are correct and that your SMTP provider allows programmatic sends.

In [1]:
from grading_utils import setup_paths

prefix = "VTC Test"
paths = setup_paths(prefix, "sample")

pdf_file = paths["pdf_file"]

Create smtp.config and provide SMTP server information

In [2]:
import configparser

config = configparser.ConfigParser()
config.read("../smtp.config")
config.sections()

email = config["default"]["email"]
password = config["default"]["password"]
smtp_server = config["default"]["smtp_server"]
smtp_port = config["default"]["smtp_port"]

KeyError: 'default'

In [ ]:
import os

# Extract paths from setup
file_name = paths["file_name"]
base_path = paths["base_path"]
base_path_marked_pdfs = paths["base_path_marked_pdfs"]
base_path_marked_scripts = paths["base_path_marked_scripts"]

In [ ]:
# load details_score_report.xlsx to df
import pandas as pd

marksDf = pd.read_excel(base_path_marked_scripts + "/details_score_report.xlsx", sheet_name="Marks")

# extract ID and Marks from df and store in a dictionary
marksDict = {}
for index, row in marksDf.iterrows():
    marksDict[row["ID"]] = row["Marks"]

# Load Performance sheet for performance reports
try:
    performanceDf = pd.read_excel(base_path_marked_scripts + "/details_score_report.xlsx", sheet_name="Performance")
    performanceDict = {}
    for index, row in performanceDf.iterrows():
        performanceDict[row["ID"]] = row["PerformanceReport"]
    print("✓ Loaded performance reports")
except Exception as e:
    print(f"⚠️ Performance sheet not found: {e}")
    performanceDict = {}

Bad pipe message: %s [b'cz\xc8\xbc^\x84\xf7\xd9\xd8XU\xd3\x81\xcc\x08\xa1\x02\xe0 .\x9e\xfci \xf8\x826\xff\xae\xc0\x9dVNV\xf2wy\xff{\xf5 \x14\xb5b\xe0\xb7j\x86\xbf,1\x00\x1a\xc0+\xc0/\xc0,\xc00\xcc\xa9\xcc\xa8\xc0\t\xc0\x13\xc0\n\xc0\x14\x13\x01\x13\x02\x13\x03\x01\x00\x05A\x00\x0b\x00\x02\x01\x00\xff\x01\x00\x01\x00\x00\x17\x00\x00\x00\x12\x00\x00\x00\x05\x00\x05\x01\x00\x00\x00\x00\x00\n\x00\x0c\x00\n\x11\xec\x00\x1d\x00\x17\x00\x18\x00\x19\x00\r\x00\x1a\x00\x18\x08\x04\x04\x03\x08\x07\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x05\x03\x06\x03\x02\x01\x02\x03\x00+\x00\x05\x04\x03']
Bad pipe message: %s [b'\x03\x003', b'\x04\xe8\x11']
Bad pipe message: %s [b'\xc0\xd7\x9a\x83AeC \xa1\x17\xcf|X\xe1R\xa6\xaa\xf6\x85u\xa9")gf\xbdweD%`\xa4\xeaM\xc7v`Z\x90\x9d\x88a\xa3\xdb\xbc\x98\x99\xbb\xa1\x92\x93UF\x97\x97\xcc\xd5\xcb\xa01\x12sx\x1b\xc23\xb0\x12\x1b\xac\xd0\xeb\x17\xbc\x85+\x93\x87\t\xafz\x11\xb7\xe2\x1c\x15\xeau\x83\xf37D\x9a\x025\xbct\xd3\xdc\xb8\xe7x\x88\xbc\xf4\x90\xa5\xeb\x9bvF\xa6\x1

✓ Loaded performance reports


# Draft the email
Please update the subject and content.

In [ ]:
subject = f"{file_name} - Your Score and Personalized Feedback"

body_template = """
<html>
<head></head>
<body>
<p>Dear Student,</p>

<p>Your total score is <strong>{marks}</strong>.</p>

{performance_report}

<p>Please find your scored script attached.</p>

<p>Regards,<br>
AI Scoring Assistant</p>
</body>
</html>
"""

Provide a function to get student email by ID.

In [ ]:
def get_email(id):
    # return "t-abcd@stu.vtc.edu.hk"
    return str(id) + "@stu.vtc.edu.hk"

In [ ]:
from grading_utils import markdown_to_html

# markdown_to_html function is now imported from grading_utils
print("✓ Using shared markdown_to_html converter")

✓ Using shared markdown_to_html converter


In [ ]:
import smtplib, ssl
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

email = config["default"]["email"]
password = config["default"]["password"]
smtp_server = config["default"]["smtp_server"]
smtp_port = config["default"]["smtp_port"]

def send_email(receiver_email, subject, body, attachment_path):
    message = MIMEMultipart()
    message["From"] = email
    message["To"] = receiver_email
    message["Subject"] = subject
    message.attach(MIMEText(body, "html"))  # Changed to HTML
    filename = os.path.basename(attachment_path)
    attachment = open(attachment_path, "rb")
    part = MIMEBase("application", "octet-stream")
    part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header("Content-Disposition", "attachment; filename= %s" % filename)
    message.attach(part)
    text = message.as_string()   
    context = ssl.create_default_context()
    with smtplib.SMTP(smtp_server, int(smtp_port)) as server:
        server.ehlo()
        server.starttls(context=context)
        server.ehlo()
        server.login(email, password, initial_response_ok=True)
        server.sendmail(email, receiver_email, text)
        server.quit()

In [ ]:
from IPython.display import display
from ipywidgets import IntProgress


f = IntProgress(min=0, max=len(marksDict))  # instantiate the bar
display(f)  # display the bar

for id in marksDict:
    marks = marksDict[id]
    performance_report_raw = performanceDict.get(id, "Performance report not available.")
    performance_report = markdown_to_html(performance_report_raw)
    body = body_template.format(marks=marks, performance_report=performance_report)
    attachment_path = base_path_marked_pdfs + str(id) + ".pdf"
    send_email(get_email(id), subject, body, attachment_path)
    f.value += 1  # signal to increment the progress bar
    # break

IntProgress(value=0, max=4)